In [1]:
#MAIN#
def MAIN_RF_MT(data_path,task_list,seed,problem_mode,main_perform,save_file_name):
    read_preprocess(data_path,task_list,seed)

    q=0
    number = list(range(0,500))    
    result_df = locals
    result_df=pd.DataFrame(index=number)   
    
    regression_col_list=['Layer','Task' , 
              'Train_MSE','Train_RMSE','Train_MAE','Train_R2','Train_Pearsonr','Train_Median_AE',            
            'Validation_MSE','Validation_RMSE','Validation_MAE','Validation_R2','Validation_Pearsonr','Validation_Median_AE',            
            'Test_MSE','Test_RMSE','Test_MAE','Test_R2','Test_Pearsonr','Test_Median_AE']            

    classification_col_list=['Layer',  'Task',
            'Train AUC', 'Train ACC','Train Balance ACC','Train F1','Train Precision', 'Train Sensitivity','Train Specificity',
            'Validation AUC','Validation ACC', 'Validation Balance ACC','Validation F1','Validation Precision','Validation Sensitivity','Validation Specificity',
              'Test_AUC', 'Test ACC', 'Test Balance ACC','Test F1',	 'Test Precision','Test Sensitivity','Test Specificity']
    
    if problem_mode=='classification':
        col_list=classification_col_list
    else:
        col_list=regression_col_list

    decision_df=pd.DataFrame()
    i=0
    for i in tqdm(range(0,10)):        
        if decision_df.empty :
            print('----training----'+str(i))  
            for label in tqdm(labels):
                train_perform(label,i,problem_mode,seed)
                for r in range(len(col_list)):
                    result_df.at[q,col_list[r]]=result_list[r]
                result_df=result_df.round(3)   
                q=q+1
                
            ####修改評估條件######
            result_df['Validation '+str(main_perform)]=result_df['Validation '+str(main_perform)].abs()
            validation_df_for_mean=result_df.loc[(result_df['Layer']==i)]                   
            validation_mean=validation_df_for_mean['Validation '+str(main_perform)].mean(axis=0)
            result_df.loc[(result_df['Layer']==i),'Mean '+str(main_perform)]=validation_mean            
            result_df['Mean '+str(main_perform)]=result_df['Mean '+str(main_perform)].round(3)
                
            if main_perform =='AUC' or 'ACC' or 'Balance ACC' or 'F1' or 'Precision' or'Recall'or'Specificity'or'R2'or'Pearsonr':
                if (i== 0) or (list(set(result_df.iloc[list(result_df.Layer== i-1) ,-1]))<list(set(result_df.iloc[list(result_df.Layer== i) ,-1]))):        
                    evaluation_NF(labels,problem_mode,i)
                else:
                    print('開始存出Excel')
                    result_df=result_df.round(3)
                    result_df=result_df.dropna()

                    result_df.to_csv('MTRFNET_'+str(save_file_name)+'_result.csv')
                    decision_df=result_df
                    #count average performance#
                    file=result_df
                    raw2mean(file,problem_mode,main_perform,save_file_name)                    

            else:
                if (i== 0) or (list(set(result_df.iloc[list(result_df.Layer== i-1) ,-1]))>list(set(result_df.iloc[list(result_df.Layer== i) ,-1]))):        
                    evaluation_NF(labels,problem_mode,i)

                else:
                    print('開始存出Excel')
                    result_df=result_df.round(3)
                    result_df=result_df.dropna()

                    result_df.to_csv('MTRFNET_'+str(save_file_name)+'_result.csv')
                    decision_df=result_df
                    #count average performance#
                    file=result_df
                    raw2mean(file,problem_mode,main_perform,save_file_name)

In [2]:
#1#
def read_preprocess(data_path,task_list,seed):
    print('-----preprocessing-----')  
    data=pd.read_csv(data_path)   
    if 'Unnamed: 0' in data:
        data=data.drop(columns=['Unnamed: 0'])
    if 'SMILES' in data:
        data=data.drop(columns=['SMILES'])
    if 'smiles' in data:
        data=data.drop(columns=['smiles'])    
    if 'mol_id' in data:
        data=data.drop(columns=['mol_id'])    

###task_list###                   
    global labels
    labels=task_list

    for i in data.index:
        for q in labels:
            try:
                if data.loc[i,q]== 'ACT':
                    data.loc[i,q]=int(1)
                if data.loc[i,q]== 'INACT':           
                    data.loc[i,q]=int(0)             
            except:
                pass   
    for label in labels:
        labels_to_drop=[l for l in labels if l !=label]
        locals()['Task_'+str(label)]  = data.drop(labels_to_drop,axis=1)  
        locals()['Task_'+str(label)]= locals()['Task_'+str(label)].dropna(subset=[label])
    for label in labels: 
        globals()['Task_'+str(label)+'_train_0'],globals()['Task_'+str(label)+'_test_0'] = train_test_split(locals()['Task_'+str(label)], test_size=0.2,random_state=seed ,stratify=locals()['Task_'+str(label)][label])
        globals()['Task_'+str(label)+'_train_0'],globals()['Task_'+str(label)+'_valid_0'] = train_test_split(globals()['Task_'+str(label)+'_train_0'], test_size=0.125,random_state=seed ,stratify=globals()['Task_'+str(label)+'_train_0'][label])

In [3]:
#2#
def train_perform(label,i,problem_mode,seed):    
    
    y_train=globals()['Task_'+str(label)+'_train_'+str(i)][label].astype('float')
    x_train=globals()['Task_'+str(label)+'_train_'+str(i)].drop(columns=label)    
    y_valid=globals()['Task_'+str(label)+'_valid_'+str(i)][label].astype('float')
    x_valid=globals()['Task_'+str(label)+'_valid_'+str(i)].drop(columns=label)   
    y_test=globals()['Task_'+str(label)+'_test_'+str(i)][label].astype('float')
    x_test=globals()['Task_'+str(label)+'_test_'+str(i)].drop(columns=label)    
    
    ##model_parameter##
    if problem_mode=='classification':
        model_parameter=RandomForestClassifier(n_estimators=500,max_features='log2',random_state=seed,n_jobs=-2)
    else:
        model_parameter=RandomForestRegressor(n_estimators=500,max_features='log2',random_state=seed,n_jobs=-2)
           
    x_train=x_train.astype(float)
    y_train=y_train.astype(float)    

    ##model##
    globals()['predictor_'+str(label)+'_'+str(i)]=model_parameter.fit(x_train, y_train)
    ###儲存模型###
    #joblib.dump(globals()['predictor_'+str(label)+'_'+str(i)], './Zebrafish_Models_withALLMorphology_ECFP6_202310_prob/predictor_'+str(label)+'_'+str(i))

    ###    
    locals()['y_train_true']=(y_train)
    locals()['y_valid_true']=(y_valid)
    locals()['y_test_true']=(y_test)

    locals()['y_train_pred']=globals()['predictor_'+str(label)+'_'+str(i)].predict(x_train)
    locals()['y_valid_pred']=globals()['predictor_'+str(label)+'_'+str(i)].predict(x_valid)
    locals()['y_test_pred']=globals()['predictor_'+str(label)+'_'+str(i)].predict(x_test)

    locals()['y_prob_train']=globals()['predictor_'+str(label)+'_'+str(i)].predict_proba(x_train)[:, 1]
    locals()['y_prob_test']=globals()['predictor_'+str(label)+'_'+str(i)].predict_proba(x_test)[:, 1]
    locals()['y_prob_valid']=globals()['predictor_'+str(label)+'_'+str(i)].predict_proba(x_valid)[:, 1]

    ##train_performances##
    global result_list
    result_list=[]
    result_list.append(int(i))
    result_list.append(label)
    
    if problem_mode == 'classification':
        
        p_list=['train','valid','test']
        for p in p_list:
            
            p_true=locals()['y_'+str(p)+'_true']
            p_prob=locals()['y_prob_'+str(p)]
            p_pred=locals()['y_'+str(p)+'_pred']
            
            locals()[str(p)+'_AUC'] =metrics.roc_auc_score(p_true,p_prob).round(3) 
            locals()[str(p)+'_ACC'] =metrics.accuracy_score(p_true, p_pred).round(3)   
            locals()[str(p)+'_BACC']=metrics.balanced_accuracy_score(p_true, p_pred).round(3)
            locals()[str(p)+'_F1'] =metrics.f1_score(p_true, p_pred).round(3)
            locals()[str(p)+'_PR'] =metrics.precision_score(p_true, p_pred).round(3)
            locals()[str(p)+'_R']=metrics.recall_score(p_true, p_pred).round(3)
            locals()[str(p)+'_S'] =metrics.recall_score(p_true, p_pred, pos_label=0).round(3)

            result_list.append(locals()[str(p)+'_AUC'])
            result_list.append(locals()[str(p)+'_ACC'])
            result_list.append(locals()[str(p)+'_BACC'])    
            result_list.append(locals()[str(p)+'_F1'])
            result_list.append(locals()[str(p)+'_PR'])
            result_list.append(locals()[str(p)+'_R'])
            result_list.append(locals()[str(p)+'_S'])        
    else:

        p_list=['train','valid','test']
        for p in p_list:
            
            p_true=locals()['y_'+str(p)+'_true']
            p_prob=locals()['y_prob_'+str(p)]
            p_pred=locals()['y_'+str(p)+'_pred']

            locals()[str(p)+'_MSE'] =metrics.mean_squared_error(p_true,p_pred).round(3) 
            locals()[str(p)+'_RMSE'] =metrics.mean_squared_error(p_true, p_pred, squared=True).round(3)   
            locals()[str(p)+'_MAE']=metrics.mean_absolute_error(p_true, p_pred).round(3)
            locals()[str(p)+'_R2'] =metrics.r2_score(p_true, p_pred).round(3)
            locals()[str(p)+'_Pearsonr'] =scipy.stats.pearsonr(p_true, p_pred).round(3)
            locals()[str(p)+'_Median_AE'] =metrics.median_absolute_error(y_train_true, y_train_pred).round(3)
	
            result_list.append(locals()[str(p)+'_MSE'])
            result_list.append(locals()[str(p)+'_RMSE'])
            result_list.append(locals()[ str(p)+'_MAE'])    
            result_list.append(locals()[str(p)+'_R2'])
            result_list.append(locals()[str(p)+'_Pearsonr'])
            result_list.append(locals()[str(p)+'_Median_AE'])

In [4]:
#3#
def evaluation_NF(labels,problem_mode,i):   
    t_v=['train','valid','test']
    ###以第0層df，當作基底增加每層特徵，索以第0層保留###
    for tv in t_v:
        print('------------'+str(tv)+'-------------')
        for Task_name in tqdm(labels):
            predict_feature=(globals()['Task_'+str(Task_name)+'_'+str(tv)+'_'+str(i)]).drop(columns=[Task_name])
            globals()['Task_'+str(Task_name)+'_'+str(tv)+'_'+str(i+1)]=globals()['Task_'+str(Task_name)+'_'+str(tv)+'_'+str(0)].copy()

            for Task_test in range(len(labels)): 
                if problem_mode=='classification':
                    prob= globals()['predictor_'+str(labels[Task_test])+'_'+str(i)].predict_proba(predict_feature)[:, 1]
                else:
                    prob= globals()['predictor_'+str(labels[Task_test])+'_'+str(i)].predict(predict_feature)

                f = 'new_feature_layer_'+str(i)+'_feature_by_'+str(labels[Task_test])
                globals()['Task_'+str(Task_name)+'_'+str(tv)+'_'+str(i+1)].loc[:,f]=prob.copy()                       
                globals()['Task_'+str(Task_name)+'_'+str(tv)+'_'+str(i+1)]=globals()['Task_'+str(Task_name)+'_'+str(tv)+'_'+str(i+1)].copy()
                globals()['Task_check'+str(Task_name)+'_'+str(tv)+'_'+str(i+1)]=globals()['Task_'+str(Task_name)+'_'+str(tv)+'_'+str(i+1)].copy()

       ##釋出不需要的variable##
    if i!=0:
        for tv in t_v:
            print('----釋出variable----'+str(tv)+'-------------')
            for Task_name in tqdm(labels):
                del globals()['Task_'+str(Task_name)+'_'+str(tv)+'_'+str(i)]

In [5]:
#4#
def raw2mean(file,problem_mode,main_perform,save_file_name):
    file['Layer']=file['Layer'].astype(int)
    file=file.dropna()

    number = list(range(0,500))
    result_mean_df=pd.DataFrame(index=number )

    regression_col_list=['Layer' , 
              'Train_MSE','Train_RMSE','Train_MAE','Train_R2','Train_Pearsonr','Train_Median_AE',            
            'Validation_MSE','Validation_RMSE','Validation_MAE','Validation_R2','Validation_Pearsonr','Validation_Median_AE',            
            'Test_MSE','Test_RMSE','Test_MAE','Test_R2','Test_Pearsonr','Test_Median_AE']            

    classification_col_list=['Layer',  
            'Train AUC', 'Train ACC','Train Balance ACC','Train F1','Train Precision', 'Train Recall','Train Specificity',
            'Validation AUC','Validation ACC', 'Validation Balance ACC','Validation F1','Validation Precision','Validation Recall','Validation Specificity',
              'Test_AUC', 'Test ACC', 'Test Balance ACC','Test F1',	 'Test Precision','Test Recall','Test Specificity']
    
    if problem_mode=='classification':
        col_list=classification_col_list
    else:
        col_list=regression_col_list
    
    col_list=classification_col_list
    col_list.append('#Mean>0.8')
    
    file_layer=file['Layer'].unique()
    file_layer=file_layer.tolist()
    mean_list=locals 
    ###各層平均###
    a=0
    for i in file_layer:

        file_layer=file['Layer'].unique()

        layer_len=len(file_layer)
        file_layer=file[file['Layer']==i]
        if problem_mode=='classification':
            file_class=file_layer.iloc[:,2:23]
        else:
            file_class=file_layer.iloc[:,2:20]            
                         
        mean_list=file_class.mean()
        mean_list=mean_list.round(3)
        mean_list=list(mean_list)
        task_number=file_class[file_class['Test_'+str(main_perform)]>=0.8]
        noft=list(task_number['Test_'+str(main_perform)])
        noft=len(noft)       
        mean_list.insert(0,i)
        mean_list.append(noft)
        for r in range(len(col_list)):
            result_mean_df.loc[a,col_list[r]]=mean_list[r]
        a=a+1

    result_mean_df=result_mean_df.dropna()    
    result_mean_df.to_csv('MTRFNET_'+str(save_file_name)+'_mean.csv')

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import metrics
import joblib
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
from resource import getrusage, RUSAGE_SELF

##########################################################
##1 random seed##
global seed
seed=8 #Ex:8

##2 檔案路徑##
#data_path=('')#Ex:'Zebrafish_Tasks_Dataset.csv'#
data_path=('Zebrafish_final202310_withMorphologyALLdata_ECFP6.csv')

##3 所有任務名稱##
#task_list=[]#Ex:['Label_name1','Label_name2','Label_name3'....]#

task_list=['LEL_MORT','LEL_YSE','LEL_AXIS','LEL_EYE','LEL_SNOU', 'LEL_JAW',
'LEL_OTIC','LEL_PE','LEL_BRAI','LEL_SOMI','LEL_PFIN','LEL_CFIN',
'LEL_PIG','LEL_CIRC', 'LEL_TRUN','LEL_SWIM','LEL_NC','LEL_TR', '18_END_LEC', 'MOR_LEC','SUBLETH_17_END_LEC', 'TOX_SCO',
'MO24', 'DP24', 'SM24', 'NC24','MORT','YSE', 'AXIS', 'EYE',
'SNOU', 'JAW', 'OTIC', 'PE', 'BRAI', 'SOMI', 'PFIN', 'CFIN',
'PIG','CIRC', 'TRUN', 'SWIM','NC','TR','MOV21', 'AUC21','METAB','MIC']

##4 結果儲存檔案名稱##
save_file_name='test' #Ex:'pk'#

###5 ####
problem_mode='classification'
#classification,regression#

###6主要評估指標####
main_perform='AUC'
# 'AUC', ' ACC',' Balance ACC',' F1',' Precision', ' Recall',' Specificity','MSE','RMSE','MAE','R2','Pearsonr','Median_AE'#         

##以下不用修改##
import time
start=time.time()
print('任務型態:'+str(problem_mode))
print('評估指標:'+str(main_perform))
MAIN_RF_MT(data_path,task_list,seed,problem_mode,main_perform,save_file_name)
end=time.time()
print('執行時間: ',end - start)
print("peak memory:", getrusage(RUSAGE_SELF).ru_maxrss / 1000 / 1000, "MB")

#目前使用的參數#
#資料比例:7:1:2
#Validation:Independet
#RF:n_estimators=500,max_features='log2'

任務型態:classification
評估指標:AUC
-----preprocessing-----


  0%|          | 0/10 [00:00<?, ?it/s]

----training----0



100%|██████████| 48/48 [02:29<00:00,  3.12s/it]


------------train-------------



100%|██████████| 48/48 [10:32<00:00, 13.17s/it]


------------valid-------------



 35%|███▌      | 17/48 [02:47<05:06,  9.88s/it]